In [ ]:
import pandas as pd
import numpy as np
import gc
import os
from datetime import date
today = date.today().strftime("%Y%m%d")

france_rep = './Vertbaudet/France'

# Compute number the INSEE number of household with children per zip and department

In [ ]:
### Load number of foyer by CODGEO via the INSEE Excel and our own calculation on the Excel columns
df_insee_nb_foy = pd.read_excel(os.path.join(france_rep,'./INSEE_data/20211008_types_menages_clean.xlsx'), usecols = ['CODGEO', 'NBR_FOYER_ENF'])
df_insee_nb_foy['NBR_FOYER_ENF'] = df_insee_nb_foy['NBR_FOYER_ENF'].astype(int)

### Load the correspondance table between CODGEO(Code Insee) and ZIP code(Code Postal) obtained on Data.gouv
df_corr = pd.read_excel(os.path.join(france_rep, './INSEE_data/correspondance-code-insee-code-postal.xlsx'), usecols = ['Code INSEE', 'Code Postal'])

### Create the df nb foyer by zip by merging
df_insee_foy_by_zip = df_insee_nb_foy.merge(df_corr, how = 'left', left_on = 'CODGEO', right_on = 'Code INSEE')

### As one Code Postal can match several CODGEO, do an aggregation
df_insee_foy_by_zip = df_insee_foy_by_zip.groupby('Code Postal').agg({'NBR_FOYER_ENF' : sum, 'Code Postal' : 'first'})

### To Excel
df_insee_foy_by_zip.to_excel(os.path.join(france_rep,'./INSEE_data/'+today+'_insee_foyer_with_children_by_zip.xlsx'))

### Create the department column
df_insee_foy_by_zip['DEPARTEMENT'] = df_insee_foy_by_zip['Code Postal'].apply(lambda x : x[:2])

### Create the df nb foyer by department
df_insee_foy_by_dep = df_insee_foy_by_zip.groupby('DEPARTEMENT').agg({'NBR_FOYER_ENF':sum})

### To Excel
df_insee_foy_by_dep.to_excel(os.path.join(france_rep,'./INSEE_data/'+today+'_insee_foyer_with_children_by_dep.xlsx'))